In [1]:
import os
import numpy as np

import sys
sys.path.insert(0, "/home/950154_customer/sungting/da_centernet_uda/centernet-uda-ang-two-stage-bimo")
from evaluation.coco_easy import Evaluator

npz_path = '/home/950154_customer/sungting/da_centernet_uda/centernet-uda-ang-two-stage-bimo/outputs/hw2itri_em_dla_itri_test/2022-06-25-17-24-51-11241153-livox_anno/npz_output'
npz_file = sorted(os.listdir(npz_path))

In [6]:
from tqdm import tqdm

## all exps <
all_exp_root_dir = {
	0: npz_path
}

all_exp = {
	0: npz_file
}

all_exp_text = {
	0: 'livox_anno vs radar_detection'
}
## all exps >


defaults = {
		'score_threshold': 0.2,
		"per_class": True}

num_classes = 3
cats = {1: {'id': 1, 'name': '"car"'},
				2: {'id': 2, 'name': '"truck/bus"'},
				3: {'id': 3, 'name': '"ped/bimo"'}}

cat_mapping = {v: i for i,
										v in enumerate(range(1, num_classes + 1))}
classes = {y: cats[x] if x in cats else ''
								for x, y in cat_mapping.items()}

# evaluator = Evaluator(**defaults)
# evaluator.classes = classes
# evaluator.num_workers = 6
# evaluator.use_rotated_boxes = True

all_result = {}
img_shape = (1142, 1142)
pixel2m = 0.175

for k, v in all_exp.items():
	# for ii, dist_logic in enumerate(logic_list):
		evaluator = Evaluator(**defaults)
		evaluator.classes = classes
		evaluator.num_workers = 20
		evaluator.use_rotated_boxes = True

		for i, exp in enumerate(tqdm(v)):
			exp_arr = np.load(os.path.join(
				all_exp_root_dir[k],
				exp
			))

			exp_arr = dict(exp_arr)

			inp = {"pred_boxes": exp_arr['pred_boxes'][None, ...],
					"pred_classes": exp_arr['pred_classes'][None, ...],
					"pred_scores": exp_arr['pred_scores'][None, ...],
					"gt_boxes": exp_arr['gt_boxes'][None, ...],
					"gt_classes": exp_arr['gt_classes'][None, ...],
					"gt_ids": [i],
					"gt_areas": None,
					"image_shape": (3, 1142, 1142)}
					
			evaluator.add_batch(**inp)
		
		result = evaluator.evaluate()
		all_result["[{}]]".format(all_exp_text[k])] = result



100%|██████████| 1372/1372 [00:52<00:00, 26.28it/s]


In [12]:
exp_arr['pred_classes'][None, ...].shape

(1, 128)

In [7]:
all_result

{'[livox_anno vs radar_detection]]': {'MSCOCO_Class_"car"/Precision/AP.50IOU': nan,
  'MSCOCO_Class_"truck/bus"/Precision/AP.50IOU': nan,
  'MSCOCO_Class_"ped/bimo"/Precision/AP.50IOU': nan,
  'MSCOCO_Precision/mAP.50IOU': nan}}